In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import creep_event_picker as cep
import obspy
from tqdm.notebook import tqdm
import h5py
import re

How many events are at each creepmeter?

In [ ]:
Creepmeter_dataframe = pd.read_csv('../../Data/DATA_tidied/creepmeter_metadata_post_standardisation_sac_codes_updated.csv',index_col=0)
Creepmeter_dataframe.drop_duplicates('Creepmeter_abbrv',inplace=True)
Creepmeter_dataframe.drop(Creepmeter_dataframe[Creepmeter_dataframe['Creepmeter_abbrv']=='XMBC'].index,inplace=True)
Creepmeter_dataframe.drop(Creepmeter_dataframe[Creepmeter_dataframe['Creepmeter_abbrv']=='TABC'].index,inplace=True)
Creepmeter_dataframe.reset_index(inplace=True,drop=True)
Creepmeter_dataframe

In [ ]:
frequency_dict = {
        1/60: '1S',    # 1 second
        1/6: '10S',  # 10 seconds
        1/2: '30S',  # 30 seconds
        1: '1T',  # 1 minute
        2: '2T',  # 2 minutes
        5: '5T',  # 5 minutes
        10: '10T',# 10 minutes
        15: '15T', # 15 minutes
        30: '30T',# 30 minutes
        60: '60T'    # 60 minutes
    }

In [ ]:
#Creepmeter_dataframe.drop(Creepmeter_dataframe[Creepmeter_dataframe['Creepmeter_abbrv']!='DU30'].index,inplace=True)

In [ ]:
Columns = ['Network','Creepmeter_full_name','Creepmeter_abbrv','Latitude','Longitude',\
           'Sampling rate, mins','ST','ET','Displacement, mm','Duration, hrs']
df_all = pd.DataFrame(columns=Columns)
e_id = 0
left_to_qc = 0
for i in tqdm(range(len(Creepmeter_dataframe))):
    creep_ortho = []
    tm_ortho = []
    if Creepmeter_dataframe['Creepmeter_abbrv'].iloc[i] in ['FCR1','GOK1','KAR1','DS30','SC30']:
        with h5py.File('../../Data/DATA_tidied/HDF5/{k}.h5'.format(k=Creepmeter_dataframe['Creepmeter_abbrv'].iloc[i]), 'r') as f:
            for key in f.keys():
                if key in ['Orthogonal']:
                    data = f[key]  # Access the dataset
                    data_keys = list(data.keys())
                    print(data_keys)
                    slip_ortho = data[data_keys[0]][:]
                    time_ortho =  data[data_keys[1]][:]
                    decoded_time = [byte_str.decode('utf-8') for byte_str in time_ortho]
                    decoded_time = pd.to_datetime(decoded_time)
                    sample_rate_ortho = (decoded_time[1]-decoded_time[0])/dt.timedelta(minutes=1)
                    smpl_rate = frequency_dict[sample_rate_ortho]


        tm_ortho, creep_ortho, upsampled = cep.interpolate_linear(decoded_time,slip_ortho,smpl_rate)  

   
    print(Creepmeter_dataframe['Creepmeter_abbrv'].iloc[i])
    
    with h5py.File('../../Data/DATA_tidied/HDF5/{k}.h5'.format(k=Creepmeter_dataframe['Creepmeter_abbrv'].iloc[i]), 'r') as f:
        # Loop through each key in the file
        keys = list(f.keys())
        f.attrs
        for key in f.keys():
            starts = []
            ends = []
            event_slip = []  
            event_duration = []
            event_vel = []
            if key not in ['Temperature','Temperature_1T','Temperature_5T','Daily_measurements','Manual_measurements','Orthogonal']:
                print("Key:", key)
                print('extracting data')
                data = f[key]  # Access the dataset
                data_keys = list(data.keys())
                print(data_keys)
                slip = data[data_keys[0]][:]
                time =  data[data_keys[1]][:]
                decoded_time = [byte_str.decode('utf-8') for byte_str in time]
                decoded_time = pd.to_datetime(decoded_time)

                match = re.search(r'_(\d*\.?\d+)mins$', key)
                if match:
                    freq_value = float(match.group(1))
                    new_suffix = frequency_dict.get(freq_value)
                    if new_suffix:
                        new_key = re.sub(r'_\d*\.?\d+mins$', f'_{new_suffix}', key)
                    else:
                        new_key = key  # or raise an error / warning
                else:
                    new_key = key  # unchanged if pattern not matched

                print(new_key)
                try:
                    df_picks = pd.read_csv("../../Data/DATA_tidied/Picks/{q}_picks.csv".format(q =new_key),index_col=0)
                except FileNotFoundError:
                    print('no events')
                    continue

                print("../../Data/DATA_tidied/Picks/{q}_picks.csv".format(q =new_key))
                print(len(df_picks))
                df_picks['Longitude'] = [Creepmeter_dataframe['Longitude'].iloc[i]]*len(df_picks)
                df_picks['Latitude'] = [Creepmeter_dataframe['Latitude'].iloc[i]]*len(df_picks)
                df_picks['Network'] = [Creepmeter_dataframe['Network'].iloc[i]]*len(df_picks)
                df_picks['Creepmeter_full_name'] = [Creepmeter_dataframe['Creepmeter_full_name'].iloc[i]]*len(df_picks)
                df_picks['Creepmeter_abbrv'] = [Creepmeter_dataframe['Creepmeter_abbrv'].iloc[i]]*len(df_picks)
                df_picks['Sampling rate, mins'] = [f[key].attrs['sampling_rate']]*len(df_picks)
                obliquity = f.attrs['obliquity']
                
                

                tm_int = decoded_time
                creep_int = slip
                
                

                if Creepmeter_dataframe['Creepmeter_abbrv'].iloc[i] not in ['CFW1','CHP1','COZ1','CPP1','CRR1','CTM1','CWC3','CWN1','SJN1','TABC',
                                                                    'WKR1','X461','XGH1','XHR1','XHR2','XHR3','XHSW','XMD1','XMM1','XMR1',
                                                                    'XPK1','XPK2','XRSW','XSC1','XSH1','XSJ2','XSJ3','XTA1','XVA1']:
                    '''if Creepmeter_dataframe['Creepmeter_abbrv'].iloc[i] == 'SC30':
                        sample_rate = frequency_dict[sample_rate]#Creepmeter_dataframe['Sampling rate, mins'].iloc[i]

            
                        tm_int, creep_int, upsampled = cep.interpolate_linear(tm_dt,creep,sample_rate)
                        tm_ortho, creep_ortho, upsampled = cep.interpolate_linear(decoded_time,slip_ortho,sample_rate=sample_rate)
                        
                        creep_int  = creep_int - creep_int[0]
                        if sample_rate =='1T':
                            record_cut = np.where(pd.to_datetime(tm_ortho)==pd.to_datetime(tm_int[0]))[0][0]
                            creep_ortho = creep_ortho[record_cut:]
                            tm_ortho = tm_ortho[record_cut:]
                        elif sample_rate == '5T':
                            record_cut = np.where(pd.to_datetime(tm_ortho)==pd.to_datetime(tm_int[-1]))[0][0]
                            creep_ortho = creep_ortho[:record_cut+1]
                            tm_ortho = tm_ortho[:record_cut+1]
                        
                            creep_ortho = creep_ortho - creep_ortho[0]
                            
                            creep_int_cor = fault_parallel_from_two_rods_array(creep_int,obliquity,creep_ortho,76)
                        else:
                            creep_int_cor = (creep_int)/np.cos(np.deg2rad(obliquity))
                    
                    
                    
                    
                    else:
                        creep_int = (creep_int)/np.cos(np.deg2rad(obliquity))'''
                    
                    creep_int = (creep_int)/np.cos(np.deg2rad(obliquity))

                
                try:
                    df_picks.drop(df_picks[df_picks['Quality_check_final'] != 'CE'].index,inplace=True)
                    print('final used')
                except:
                    df_picks.drop(df_picks[df_picks['Quality_check_new'] != 'CE'].index,inplace=True)
                    print('new used')
                df_picks.reset_index(inplace=True,drop=True)
                
                for j in tqdm(range(len(df_picks))):
                    # Find the index of the closest datetime
                    try:
                        nearest_s = tm_int[np.argmin(np.abs(np.array(tm_int) - np.datetime64(df_picks.ST_final.iloc[j])))]
                        nearest_e = tm_int[np.argmin(np.abs(np.array(tm_int) - np.datetime64(df_picks.ET_final.iloc[j])))]
                        
                    except:
                        nearest_s = tm_int[np.argmin(np.abs(np.array(tm_int) - np.datetime64(df_picks.ST_new.iloc[j])))]
                        nearest_e = tm_int[np.argmin(np.abs(np.array(tm_int) - np.datetime64(df_picks.ET_new.iloc[j])))]
                        
                    
                    starts.append(nearest_s)
                    ends.append(nearest_e)
                    boolarr = np.logical_and(np.array(tm_int)>=nearest_s,np.array(tm_int)<=nearest_e)
                    event_creep = creep_int[boolarr]
                    event_time = tm_int[boolarr]
                    event_slip.append(event_creep[-1] - event_creep[0])
                    event_duration.append((pd.to_datetime(event_time[-1]) - pd.to_datetime(event_time[0]))/dt.timedelta(hours=1))
                df_picks['Displacement, mm'] = event_slip
                df_picks['Duration, hrs'] = event_duration
                df_picks['Max Velocity, m/s'] = event_vel
                df_picks['ST'] = starts
                df_picks['ET'] = ends
                
                before = len(df_all)
                df_all = pd.concat([df_all, df_picks[Columns]], ignore_index=True)
                after = len(df_all)
                print(f"Rows before: {before}, Rows after: {after}, Rows added: {after-before}")
print('Left to QC:',left_to_qc)

  0%|          | 0/74 [00:00<?, ?it/s]

BAL1
Key: BAL1_1.0mins
extracting data
['Slip_BAL1_1.0mins', 'Time_BAL1_1.0mins']
BAL1_1T
../../Data/DATA_tidied/Picks/BAL1_1T_picks.csv
3
final used


  0%|          | 0/2 [00:00<?, ?it/s]

0.07678758580221956
0.015126577052768653
Rows before: 0, Rows after: 2, Rows added: 2
C461
Key: C461_10.0mins
extracting data
['Slip_C461_10mins', 'Time_C461_10mins']


/var/folders/56/47k8q3q1449cq8kwzkz5r8ch0000gr/T/ipykernel_18419/3299252034.py:174: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat([df_all, df_picks[Columns]], ignore_index=True)


...

  0%|          | 0/97 [00:00<?, ?it/s]

0.07000000000000028
0.020000000000000462
0.020000000000003126
0.030000000000001137
0.030000000000001137
0.060000000000002274
0.05999999999999517
0.020000000000003126
0.020000000000003126
0.020000000000010232
0.01999999999999602
0.01999999999999602
0.0800000000000125
0.020000000000010232
0.010000000000005116
0.060000000000002274
0.010010000000022501
0.020010000000013406
0.020010000000013406
0.020010000000013406
0.020010000000013406
0.030020000000007485
0.020010000000013406
0.020010000000013406
0.020009999999984984
0.020010000000013406
0.030000000000001137
0.020009999999984984
0.010010000000022501
0.02001999999998816
0.03000000000002956
0.019989999999950214
0.03000000000002956
0.020020000000045002
0.020020000000045002
0.03000000000002956
0.020020000000045002
0.050010000000042965
0.020020000000045002
0.020020000000045002
0.02001999999998816
0.019990000000007058
0.030020000000035907
1.8799999999999955
0.020010000000013406
0.020020000000045002
0.040009999999995216
0.020010000000013406
0.029

  0%|          | 0/48 [00:00<?, ?it/s]

0.06899999999999906
0.004000000000001336
0.004000000000001336
0.004000000000001336
0.004000000000001336
0.004000000000001336
0.008000000000002672
0.06500000000000128
0.0070000000000014495
0.004000000000001336
0.004000000000001336
0.010999999999999233
0.03999999999999915
0.0940000000000012
0.10999999999999943
0.029000000000003467
0.008000000000002672
0.008000000000002672
0.006999999999997897
0.04700000000000415
0.008000000000002672
0.008000000000002672
0.0040000000000048885
0.006999999999997897
0.0040000000000048885
0.0040000000000048885
0.0040000000000048885
0.0040000000000048885
0.0040000000000048885
0.3709999999999951
0.014000000000002899
0.03999999999999915
0.0040000000000048885
0.0040000000000048885
0.03999999999999915
0.0040000000000048885
0.0040000000000048885
0.0040000000000048885
0.0040000000000048885
0.134999999999998
0.0040000000000048885
0.0040000000000048885
0.007000000000005002
0.0040000000000048885
0.0040000000000048885
0.0040000000000048885
0.0040000000000048885


ValueError: zero-size array to reduction operation maximum which has no identity

In [20]:
event_creep

array([47.596])

In [21]:
nearest_e


Timestamp('2023-02-18 23:59:00')

In [22]:
nearest_s

Timestamp('2023-02-18 23:59:00')

In [ ]:
Creepmeter_dataframe['Creepmeter_abbrv'].iloc[i]

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_all.loc[df_all['Creepmeter_abbrv']=='DU30'])

In [ ]:
#df_all.dropna(inplace=True)
df_all.sort_values(by='ST',inplace=True)
df_all.reset_index(inplace=True,drop=True)
df_all

In [ ]:
event_id = []
for i in range(len(df_all)):
    event_id.append("{:04d}".format(i) + '_{k}'.format(k=df_all.Creepmeter_abbrv.iloc[i]))
df_all['Event_id'] = event_id
df_all

In [ ]:
df_all.to_csv("../../Data/DATA_tidied/Picks/All_picks_23_APR_2025.csv")

In [ ]:
df_all['Creepmeter_abbrv'].unique()

Testing for adding in orthogonal component

In [ ]:
def fault_parallel_from_two_rods_array(d1, theta1_deg, d2, theta2_deg):
    """
    Vectorised version to compute fault-parallel slip from two rods over a time series.

    d1, d2: arrays of displacement time series (same shape)
    theta1_deg, theta2_deg: angles from fault-parallel (scalars, in degrees)
    
    Returns: array of s_parallel (same shape as d1 and d2)
    """
    theta1 = np.radians(theta1_deg)
    theta2 = np.radians(theta2_deg)

    sin1 = np.sin(theta1)
    sin2 = np.sin(theta2)
    cos1 = np.cos(theta1)
    cos2 = np.cos(theta2)

    numerator = d2 * sin1 - d1 * sin2
    denominator = cos2 * sin1 - cos1 * sin2

    if np.isclose(denominator, 0).any():
        raise ValueError("Denominator close to zero — angles may be degenerate or too similar.")

    s_parallel = numerator / denominator
    return s_parallel

In [ ]:
abbrv = 'FCR1'

In [ ]:
if abbrv in ['FCR1','GOK1','KAR1','DS30','SC30']:
    with h5py.File('../../Data/DATA_tidied/HDF5/{k}.h5'.format(k=abbrv), 'r') as f:
        for key in f.keys():
            if key in ['Orthogonal']:
                data = f[key]  # Access the dataset
                data_keys = list(data.keys())
                print(data_keys)
                slip_ortho = data[data_keys[0]][:]
                time_ortho =  data[data_keys[1]][:]
                decoded_time = [byte_str.decode('utf-8') for byte_str in time_ortho]
                decoded_time = pd.to_datetime(decoded_time)


    

    path = "../../Data/DATA_tidied/SAC/{q}.SAC".format(q='FCR1_1T')#Creepmeter_dataframe.File_code.iloc[i])
    st = obspy.read(path)
    tr = st[0]
    tr.interpolate(sampling_rate=tr.stats.sampling_rate)
    tm = tr.times("utcdatetime")
    creep = tr.data
    sample_rate =  round(tr.stats.delta)/60
    tm_dt = []
    for w in range(len(tm)):
        tm_new = tm[w].datetime
        tm_dt.append(tm_new)

    sample_rate = frequency_dict[sample_rate]#Creepmeter_dataframe['Sampling rate, mins'].iloc[i]

    
    tm_int, creep_int, upsampled = cep.interpolate_linear(tm_dt,creep,sample_rate)
    tm_ortho, creep_ortho, upsampled = cep.interpolate_linear(decoded_time,slip_ortho,sample_rate=sample_rate)
    
    creep_int  = creep_int - creep_int[0]
    creep_ortho = creep_ortho[:-1] - creep_ortho[0]
    creep_int_cor = fault_parallel_from_two_rods_array(creep_int,30,creep_ortho,90)

In [ ]:
record_cut

In [ ]:
plt.figure()
plt.plot(tm_ortho, creep_ortho)
plt.plot(tm_int,creep_int)
plt.plot(tm_int,creep_int_cor)
plt.show()

In [ ]:
tm_int[-1]

In [ ]:
tm_ortho[-1]

In [ ]:
elif 'Quality_check' in df_picks.columns:
            left_to_qc +=1
            #print(Creepmeter_dataframe["File_code"].iloc[i],'old QC')
            df_picks.drop(df_picks[df_picks['Quality_check'] != 'CE'].index,inplace=True)
            df_picks.reset_index(inplace=True,drop=True)
            print(len(df_picks))
            for j in range(len(df_picks)):
                #print(j,df_picks.ST_new.iloc[j],df_picks.ET_new.iloc[j])
                #print('old QC')
                boolarr = np.logical_and(np.array(tm_int)>=np.datetime64(df_picks.ST.iloc[j]),\
                                        np.array(tm_int)<=np.datetime64(df_picks.ET.iloc[j]))
                event_creep = creep_int[boolarr]
                event_time = tm_int[boolarr]
                e_id +=1
                event_code.append('{k}'.format(k=Creepmeter_dataframe.Creepmeter_abbrv.iloc[i]) + "{:04d}".format(e_id))
                event_slip.append(event_creep[-1] - event_creep[0])
                event_duration.append((pd.to_datetime(event_time[-1]) - pd.to_datetime(event_time[0]))/dt.timedelta(hours=1))
            df_picks['Displacement, mm'] = event_slip
            df_picks['Duration, hrs'] = event_duration
            df_picks['Event_id'] = event_code
            df_picks['ST'] = df_picks['ST'].astype(str).str[0:19]
            df_picks['ET'] = df_picks['ET'].astype(str).str[0:19]
            df_picks['ST'] = pd.Series(pd.to_datetime(df_picks['ST'])).\
                dt.round(Creepmeter_dataframe['Sampling rate, mins'].iloc[i])
            df_picks['ET'] = pd.Series(pd.to_datetime(df_picks['ET'])).\
                dt.round(Creepmeter_dataframe['Sampling rate, mins'].iloc[i])
        
        else:
            print(len(df_picks))
            print(Creepmeter_dataframe["File_code"].iloc[i],'no QC')
            left_to_qc +=1
            for j in range(len(df_picks)):
                #print('old QC')
                boolarr = np.logical_and(np.array(tm_int)>=np.datetime64(df_picks.ST.iloc[j]),\
                                        np.array(tm_int)<=np.datetime64(df_picks.ET.iloc[j]))
                event_creep = creep_int[boolarr]
                event_time = tm_int[boolarr]
                e_id +=1
                event_code.append('{k}'.format(k=Creepmeter_dataframe.Creepmeter_abbrv.iloc[i]) + "{:04d}".format(e_id))
                event_slip.append(event_creep[-1] - event_creep[0])
                event_duration.append((pd.to_datetime(event_time[-1]) - pd.to_datetime(event_time[0]))/dt.timedelta(hours=1))
            df_picks['Displacement, mm'] = event_slip
            df_picks['Duration, hrs'] = event_duration
            df_picks['Event_id'] = event_code
            df_picks['ST'] = df_picks['ST'].astype(str).str[0:19]
            df_picks['ET'] = df_picks['ET'].astype(str).str[0:19]
            df_picks['ST'] = pd.Series(pd.to_datetime(df_picks['ST'])).\
                dt.round(Creepmeter_dataframe['Sampling rate, mins'].iloc[i])
            df_picks['ET'] = pd.Series(pd.to_datetime(df_picks['ET'])).\
                dt.round(Creepmeter_dataframe['Sampling rate, mins'].iloc[i])

In [ ]:
%matplotlib qt

In [ ]:
plt.close('all')
with h5py.File('../../Data/DATA_tidied/HDF5/DU30.h5', 'r') as f:
        # Loop through each key in the file
        keys = list(f.keys())
        f.attrs
        for key in f.keys():
            starts = []
            ends = []
            event_slip = []  
            event_duration = []
            #event_code = []
            if key not in ['Temperature','Temperature_1T','Temperature_5T','Daily_measurements','Manual_measurements','Orthogonal']:
                print("Key:", key)
                print('extracting data')
                data = f[key]  # Access the dataset
                data_keys = list(data.keys())
                print(data_keys)
                slip = data[data_keys[0]][:]
                time =  data[data_keys[1]][:]
                decoded_time = [byte_str.decode('utf-8') for byte_str in time]
                decoded_time = pd.to_datetime(decoded_time)
        
            plt.figure()
            plt.plot(decoded_time,slip,label=key)
            plt.legend()
            plt.show()